In [1]:
# package(s) related to time, space and id
import datetime

# Used for mathematical functions
import math
import platform
import time

import matplotlib.pyplot as plt

# Used for making the graph to visualize our problem
import networkx as nx

# package(s) for data handling
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import itertools
import tqdm

import plotly.express as px
from plotly.subplots import make_subplots
# spatial libraries
import pyproj
import shapely.geometry

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
from simplekml import Kml, Style



#### define function

In [2]:
### Limit Speed
def f(x):
    ##As = Bs * T
    #Ac = Bc * h
    #As/Ac
    res = 1- 0.0633 + 0.5*x**2 -1.5 * x **(2/3)
    return res
 
def dfdx(x):
    res1 = x -  x**(-1/3)
    return res1

# Newton-Raphson Algorithm
max_iter = 5  # Max iterations
tol = 1E-15  # Tolerance
i = 0  # Iteration counter
x0 = 0.8  # Initial guess
xi_1 = x0
print('Iteration ' + str(i) + ': x = ' + str(x0) + ' f(x) =' + str(f(x0)))
# Iterating until either the tolerance or max iterations is met
while abs(f(xi_1)) > tol or i > max_iter:
    i = i + 1
    xi = xi_1-f(xi_1)/dfdx(xi_1)  # Newton-Raphson equation
    print('Iteration '+ str(i) + ': x = ' + str(xi) + ' f(x) = ' + str(f(xi)))
    xi_1 = xi
    
def get_Vlimit (xi_1):
    Vlim = xi_1 * np.sqrt(9.81*h)
    return Vlim

def get_Vs():
    if V <= Vlim:
        Vs = V
    else:
        Vs = Vlimit
    return Vs    
    
#### Froude number
def get_Fr (V, h, Ur):
    Frh = (V + Ur) / np.sqrt(h * 9.81)
    return Frh


# Return current speed 
from scipy.misc import derivative

def g(x):
    Ur = ((2*Vs*x + (x**2))/(2*9.81*h)) - (x /(Vs + x)) + 28/1200
    return Ur

# The Newton-Raphson function
def NewtonRaphson(g, xi):
    x = xi
    while abs(g(x)) > 1e-6:
        gx = g(x)
        dgx = derivative(g, x, dx = 1e-6)
        x = x - (gx/dgx)
    return x


####Rf
def get_Rf(Bs, T, Ls):
    # get cf
    Re = ((Vs+Ur)*Ls)/ (10**(-6))
    cf = 0.075 /((np.log10(Re)-2)**2)
   
    Rf = 1.2 * 0.5 *((Vs+0.067)**2) * cf * (Bs *Ls + 2 * Ls * T)
    return  Rf

#####Rp
def get_Rp():
    Rp = 0.5 *(Vs**2) * 0.1 * (Bs * T)
    return Rp

### z
def get_z():
    Frh =  Vs/ np.sqrt(h*9.81)
    
    if Ac/As <= 5.45*h + 0.56*As-17.68:  #wide or narrow
        z = (((Vs + 0.067)**2)-(0.067**2))/(2*9.81)*(-0.00002*(Bc/Bs)**3 + 0.0062*(Bc/Bs)**2-0.007*(Bc/Bs)+0.9927)
        
    else:#very wide
        z = 1.36 * (Bs*Ls*T)/Ls**2 * Frh**2/ np.sqrt(1-Frh**2)
    return z
##### Rz
def get_Rz():
    Rz = 9.81 *0.0499 * 0.2 * (Bs * T)
    return Rz

####Rtot
def get_Rtot(Rp, Rf, Rz):
    
    Rtot =  Rp + Rf + Rz
    return Rtot

# Power estimation 
# BHP > DHP > EHP
# 1) Effective horse power Pe (Power needed to overcome the total drag)
def get_Pe ():
    Pe = Vs * Rtot 
    return Pe

# 3) Brake horse power or Propulsion power Pb (Required motor power)
def get_Pb ():
    Pb = (Vs*Rtot)/(0.55*0.91*1.07)
    return Pb

# 2) Delivered horse power Pd (Power delivered to the propeller shaft)
def get_Pd (Pb):
    Pd = 0.95*Pb
    return Pd

Iteration 0: x = 0.8 f(x) =-0.03596081401913054
Iteration 1: x = 0.6702793506046221 f(x) = 0.012491196700400664
Iteration 2: x = 0.696722764556478 f(x) = 0.0005448963906875548
Iteration 3: x = 0.6979861804150457 f(x) = 1.228482741533199e-06
Iteration 4: x = 0.6979890417181043 f(x) = 6.297407040278813e-12
Iteration 5: x = 0.697989041732772 f(x) = -4.440892098500626e-16


#### input value for Urel

In [3]:
# M6 motor vessel
h = np.linspace(3,10,71)  # water depth from 3 m to 10 m is a suitable range for the Ductch waterways 
h_more = np.linspace(3,10,29)
V_more = np.linspace(0,6,25)
Bs = [9.5]
Bc= [150] 
V = [3.5]
T = [2]
Ls = [85]

#### prepare input matrix for calculation

In [4]:
V_more = np.linspace(0,6,25)
V_more

array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 ,
       2.75, 3.  , 3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75, 5.  , 5.25,
       5.5 , 5.75, 6.  ])

In [5]:
h_more = np.linspace(3,10,29)
h_more

array([ 3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.25,  4.5 ,  4.75,  5.  ,
        5.25,  5.5 ,  5.75,  6.  ,  6.25,  6.5 ,  6.75,  7.  ,  7.25,
        7.5 ,  7.75,  8.  ,  8.25,  8.5 ,  8.75,  9.  ,  9.25,  9.5 ,
        9.75, 10.  ])

In [6]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product( h_more, Bs, Bc, V_more, T, Ls))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"h_more": item[0],"Bs": item[1], "Bc": item[2], "V_more": item[3], "T": item[4], "Ls": item[5]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.head(50)

,h_more,Bs,Bc,V_more,T,Ls
0,3.00,9.5,150,0.00,2,85
1,3.00,9.5,150,0.25,2,85
2,3.00,9.5,150,0.50,2,85
3,3.00,9.5,150,0.75,2,85
4,3.00,9.5,150,1.00,2,85
5,3.00,9.5,150,1.25,2,85
6,3.00,9.5,150,1.50,2,85
7,3.00,9.5,150,1.75,2,85
8,3.00,9.5,150,2.00,2,85
9,3.00,9.5,150,2.25,2,85


#### use the defined function and get output table

In [7]:
results = []

for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    h = row['h_more']
    Bs = row['Bs']
    Bc = row['Bc']
    V = row['V_more']
    T = row['T']
    Ls = row['Ls']

    res = f(x)
    res1 = dfdx(x) 
    Frh = get_Fr(V, h)
    r, Vadj = get_Vadj(Bs, Bc, V, T, h, Ls, swf)
    Ur = get_Ur(Bs, Bc, V, T, h)
    Urel, Unarrow, Ushallow, Vadj = get_Urel (h, T, Ur, Vadj, swf)
    Rf = get_Rf(Bs, Urel, T, Ls)
    Rp = get_Rp(Vadj, T)
    Rz = get_Rz (Bs, Bc, T, h, V, Ur, Ls)
    Rtot = get_Rtot(Rp, Rf, Rz)
    Pe = get_Pe ()
    Pd = get_Pd (Urel, Rtot)
    Pb = get_Pb (Pd)
    E = get_E (Vadj, Pb)

        
    result = {}
    result.update(row)
    result['h/T'] = r
    result['Fr'] = Fr
    result['Vadj'] = Vadj
    result['Ur'] = Ur
    result['Urel'] = Urel
    result['Unarrow'] = Unarrow
    result['Ushallow'] = Ushallow
    result['Rf'] = Rf
    result['Rp'] = Rp
    result['Rz'] = Rz
    result['Rtot']= Rtot
    result['Pe'] = Pe
    result['Pb'] = Pb
    result['Pd'] = Pd
    result['E'] = E

    results.append(result)

plot_df = pd.DataFrame(results)

plot_df.head(50)

0it [00:00, ?it/s]


NameError: name 'x' is not defined

#### select desired variables for plotting

In [ ]:
selected = plot_df.query('V_more == 4')



plt.plot(selected['h_more'], selected['E'], 'black', linewidth = 3.0)

plt.ylabel('Ri (kN)')
plt.legend(['E',])
plt.grid()
plt.xlim(3, 10)
plt.ylim(0,200)
plt.title('Energy kW/km');

In [ ]:
selected_v_4 = plot_df.query('V_more == 4.5')
selected_v_2 = plot_df.query('V_more == 2.5')
plt.plot(selected_v_4['h/T'], selected_v_4['Vadj'], 'darkgreen', linewidth = 3.0)
plt.plot(selected_v_2['h/T'], selected_v_2['Vadj'], 'lightgreen', linewidth = 3.0)
plt.xlabel('h/T')
plt.ylabel('Vadj (m/s)')

plt.grid()
plt.xlim(0.9375)
plt.ylim(0)
plt.legend(['V =4.5 m/s','V =2.5 m/s'])
plt.title('Adjusted velocity');

In [ ]:
selected_v_4 = plot_df.query('V_more == 4.5')
selected_v_2 = plot_df.query('V_more == 2.5')
plt.plot(selected_v_4['h/T'], selected_v_4['Ushallow'], 'darkgreen', linewidth = 3.0)
plt.plot(selected_v_2['h/T'], selected_v_2['Ushallow'], 'lightgreen', linewidth = 3.0)
plt.xlabel('h/T')
plt.ylabel('Ushallow (m/s)')

plt.grid()
plt.xlim(0.9375)
plt.ylim(0)
plt.legend(['V =4.5 m/s','V =2.5 m/s'])
plt.title('Shallow velocity');

In [ ]:
selected_v_4 = plot_df.query('V_more == 4.5')
selected_v_2 = plot_df.query('V_more == 2.5')
plt.plot(selected_v_4['h/T'], selected_v_4['Unarrow'], 'darkgreen', linewidth = 3.0)
plt.plot(selected_v_2['h/T'], selected_v_2['Unarrow'], 'lightgreen', linewidth = 3.0)
plt.xlabel('h/T')
plt.ylabel('Unarrow (m/s)')

plt.grid()
plt.xlim(0.9375)
plt.ylim(0)
plt.legend(['V =4.5 m/s','V =2.5 m/s'])
plt.title('Narrow velocity');

In [ ]:
selected_v4 = plot_df.query('V_more == 4.5')
selected_v2 = plot_df.query('V_more == 2.5')
plt.plot(selected_v4['h/T'], selected_v4['Urel'], 'darkgreen', linewidth = 3.0)
plt.plot(selected_v2['h/T'], selected_v2['Urel'], 'lightgreen', linewidth = 3.0)
plt.xlabel('h/T')
plt.ylabel('Urel (m/s)')

plt.grid()
plt.xlim(0.9375)
plt.ylim(0)
plt.legend(['V =4.5 m/s','V =2.5 m/s'])
plt.title('Relative velocity');

In [ ]:
selected = plot_df.query('V_more == 4')



plt.plot(selected['h_more'], selected['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected['h_more'], selected['Rf'], 'red', linewidth = 3.0)
plt.plot(selected['h_more'], selected['Rp'], 'green', linewidth = 3.0)
plt.plot(selected['h_more'], selected['Rz'], 'orange', linewidth = 3.0)
plt.xlabel('h (m)')
plt.ylabel('Ri (kN)')
plt.legend(['Rtot','Rf','Rp','Rz',])
plt.grid()
plt.xlim(3, 10)
plt.ylim()
plt.title('Resistance components at V = 4 m/s');

In [ ]:
selected = plot_df.query('V_more == 4')


plt.plot(selected['h_more'], selected['Pe'], 'purple', linewidth = 2.0)
plt.plot(selected['h_more'], selected['Pd'], 'gray', linewidth = 2.0)
plt.plot(selected['h_more'], selected['Pb'], 'pink', linewidth = 2.0)

plt.xlabel('h (m)')
plt.ylabel('Power (kW)')
plt.legend(['Pe','Pd','Pb'])
plt.grid()
plt.xlim(3,10)
plt.ylim(0)
plt.title('Power estimation at V = 4 m/s');

In [ ]:
selected = plot_df.query('V_more == 4.5')


plt.plot(selected['h/T'], selected['Pe'], 'purple', linewidth = 2.0)
plt.plot(selected['h/T'], selected['Pd'], 'gray', linewidth = 2.0)
plt.plot(selected['h/T'], selected['Pb'], 'pink', linewidth = 2.0)

plt.xlabel('h/T')
plt.ylabel('Power (kW)')
plt.legend(['Pe','Pd','Pb'])
plt.grid()
plt.xlim(1)
plt.ylim(0)
plt.title('Power estimation at V = 4.5 m/s');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3')
selected_h_3_5 = plot_df.query('h_more == 3.5')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')


plt.figure(dpi=400)
plt.plot(selected_h_3['V_more'],selected_h_3['Rf'], 'navy', linewidth = 3.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rf'], 'r', linewidth = 1.0, linestyle = 'dashed')
plt.plot(selected_h_4['V_more'],selected_h_4['Rf'], 'maroon', linewidth = 1.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rf'], 'red', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rf'], 'indianred', linewidth = 1)

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Rf (kN)')
plt.legend(['Rf~3m','Rf~3.5m','Rf~4m','Rf~5m','Rf~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim(0)
plt.title('Frictional Resistance');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4.5')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.5')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')


plt.figure(dpi=400)
plt.plot(selected_h_3['V_more'],selected_h_3['Rp'], 'navy', linewidth = 4.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rp'], 'r', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rp'], 'green', linewidth = 1.0, linestyle = 'dashed')
plt.plot(selected_h_5['V_more'],selected_h_5['Rp'], 'green', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rp'], 'darkgreen', linewidth = 1)

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Rp (kN)')
plt.legend(['Rp~3m','Rp~3.5m','Rp~4m','Rp~5m','Rp~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Pressure Resistance');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4.5')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.5')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')


plt.figure(dpi=100)
plt.plot(selected_h_3['V_more'],selected_h_3['Rz'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rz'], 'r', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rz'], 'orange', linewidth = 1.0, linestyle = 'dashed')
plt.plot(selected_h_5['V_more'],selected_h_5['Rz'], 'orange', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rz'], 'darkorange', linewidth = 1)

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Rz (kN)')
plt.legend(['Rz~3m','Rz~3.5m','Rz~4m','Rz~5m','Rz~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Rz');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 3.5')
selected_h_3_2 = plot_df.query('h_more == 3.25 & V_more <= 3.8')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 3.9')
selected_h_4 = plot_df.query('h_more == 4 & V_more <= 4')
selected_h_4_2 = plot_df.query('h_more == 4.25 & V_more <= 4.2')
selected_h_5 = plot_df.query('h_more == 5 & V_more <= 4.6')
selected_h_5_5= plot_df.query('h_more == 5.5 & V_more <= 4.8')
selected_h_8 = plot_df.query('h_more == 8 & V_more <= 4.8')
selected_h_5 = plot_df.query('h_more == 5 & V_more <= 4.8')
selected_h_10 = plot_df.query('h_more == 10 & V_more <= 4.8')



plt.plot(selected_h_3['V_more'],selected_h_3['Rtot'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_2['V_more'],selected_h_3_2['Rtot'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rtot'], 'r', linewidth = 1.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rtot'], 'darkviolet', linewidth = 1.0)
plt.plot(selected_h_4_2['V_more'],selected_h_4_2['Rtot'], 'darkviolet', linewidth = 1.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rtot'], 'violet', linewidth = 1.0)
plt.plot(selected_h_5_5['V_more'],selected_h_5_5['Rtot'], 'violet', linewidth = 1.0)
plt.plot(selected_h_8['V_more'],selected_h_8['Rtot'], 'rebeccapurple', linewidth = 1)
plt.plot(selected_h_10['V_more'],selected_h_10['Rtot'], 'rebeccapurple', linewidth = 1)

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Rtot (kN)')
plt.legend(['Rtot~3m','Rtot~3.5m','Rtot~4m','Rtot~5m','Rtot~6m'])
plt.xlim(0,5)
plt.ylim()
plt.grid()
plt.title('Total Resistance');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3')



plt.plot(selected_h_3['V_more'],selected_h_3['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Rf'], 'red', linewidth = 4.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Rp'], 'green', linewidth = 3.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Rz'], 'orange', linewidth = 2)

plt.xlabel('V (m/s)')
plt.ylabel('R (kN)')
plt.legend(['Rtot~3m','Rf~3m','Rp~3m','Rz~3m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Resistance components at 3 m');

In [ ]:

selected_h_3_5 = plot_df.query('h_more == 3.5')

plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rf'], 'red', linewidth = 4.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rp'], 'green', linewidth = 3.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rz'], 'orange', linewidth = 2)

plt.xlabel('V (m/s)')
plt.ylabel('R (kN)')
plt.legend(['Rtot~3.5m','Rf~3.5m','Rp~3.5m','Rz~3.5m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Resistance components at 3.5 m');

In [ ]:
selected_h_4 = plot_df.query('h_more == 4')



plt.plot(selected_h_4['V_more'],selected_h_4['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rf'], 'red', linewidth = 4.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rp'], 'green', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rz'], 'orange', linewidth = 2)


plt.xlabel('V (m/s)')
plt.ylabel('R (kN)')
plt.legend(['Rtot~4m','Rf~4m','Rp~4m','Rz~4m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Resistance components at 4 m');

In [ ]:
selected_h_5 = plot_df.query('h_more == 5')



plt.plot(selected_h_5['V_more'],selected_h_5['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rf'], 'red', linewidth = 4.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rp'], 'green', linewidth = 3.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rz'], 'orange', linewidth = 2)


plt.xlabel('V (m/s)')
plt.ylabel('R (kN)')
plt.legend(['Rtot~5m','Rf~5m','Rp~5m','Rz~5m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Resistance components at 5 m');

In [ ]:
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_6['V_more'],selected_h_6['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rf'], 'red', linewidth = 4.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rp'], 'green', linewidth = 3.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rz'], 'orange', linewidth = 2)


plt.xlabel('V (m/s)')
plt.ylabel('R (kN)')
plt.legend(['Rtot~6m','Rf~6m','Rp~6m','Rz~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Resistance components at 6 m');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3')


plt.figure(dpi=400)
plt.plot(selected_h_3['V_more'],selected_h_3['Pe'], 'pink', linewidth = 3.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Pb'], 'purple', linewidth = 3.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Pd'], 'blueviolet', linewidth = 3.0)


plt.xlabel('V (m/s)')
plt.ylabel('Power (kW)')
plt.legend(['Pe~3m','Pb~3m','Pd~3m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Power at 3 m');

In [ ]:
selected_h_3_5 = plot_df.query('h_more == 3.5')


plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pe'], 'pink', linewidth = 3.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pb'], 'purple', linewidth = 3.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pd'], 'blueviolet', linewidth = 3.0)


plt.xlabel('V (m/s)')
plt.ylabel('Power (kW)')
plt.legend(['Pe~3.5m','Pb~3.5m','Pd~3.5m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Power at 3.5 m');

In [ ]:
selected_h_4 = plot_df.query('h_more == 4')



plt.plot(selected_h_4['V_more'],selected_h_4['Pe'], 'pink', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Pb'], 'purple', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Pd'], 'blueviolet', linewidth = 3.0)


plt.xlabel('V (m/s)')
plt.ylabel('Power (kW)')
plt.legend(['Pe~4m','Pb~4m','Pd~4m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Power at 4 m');

In [ ]:
selected_h_5 = plot_df.query('h_more == 5')



plt.plot(selected_h_5['V_more'],selected_h_5['Pe'], 'pink', linewidth = 3.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Pb'], 'purple', linewidth = 3.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Pd'], 'blueviolet', linewidth = 3.0)


plt.xlabel('V (m/s)')
plt.ylabel('Power (kW)')
plt.legend(['Pe~5m','Pb~5m','Pd~5m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim(0)
plt.title('Power at 5 m');

In [ ]:
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_6['V_more'],selected_h_6['Pe'], 'pink', linewidth = 3.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Pb'], 'purple', linewidth = 3.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Pd'], 'blueviolet', linewidth = 3.0)


plt.xlabel('V (m/s)')
plt.ylabel('Power (kW)')
plt.legend(['Pe~6m','Pb~6m','Pd~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim(0)
plt.title('Power at 6 m');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4.5')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.5')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_3['V_more'],selected_h_3['Pb'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pb'], 'red', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Pb'], 'pink', linewidth = 2.5, linestyle = 'dotted')
plt.plot(selected_h_5['V_more'],selected_h_5['Pb'], 'pink', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Pb'], 'pink', linewidth = 1, linestyle = 'dashed')

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Pb (kW)')
plt.legend(['Pb~3m','Pe~3.5m','Pb~4m','Pb~5m','Pb~6m'])
plt.xlim(0, 5)
plt.ylim()
plt.grid()
plt.title('Pb');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 3.5')
selected_h_3_2 = plot_df.query('h_more == 3.25 & V_more <= 3.7')
#selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 3.9')
#selected_h_4 = plot_df.query('h_more == 4 & V_more <= 4')
selected_h_4_2 = plot_df.query('h_more == 4.25 & V_more <= 4.2')
selected_h_5 = plot_df.query('h_more == 5 & V_more <= 4.55')
selected_h_5_5= plot_df.query('h_more == 5.5 & V_more <= 4.77')
selected_h_8 = plot_df.query('h_more == 8 & V_more <= 4.9' )
selected_h_10 = plot_df.query('h_more == 10 & V_more <= 4.9')

plt.figure(dpi=300)
plt.figure(figsize=(8,7))
plt.plot(selected_h_3['V_more'],selected_h_3['Rtot'], 'magenta', linewidth = 1.0)
plt.plot(selected_h_3_2['V_more'],selected_h_3_2['Rtot'], 'aqua', linewidth = 1.0)
#plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pd'], 'skyblue', linewidth = 1.0)
#plt.plot(selected_h_4['V_more'],selected_h_4['Pd'], 'skyblue', linewidth = 1.0)
plt.plot(selected_h_4_2['V_more'],selected_h_4_2['Rtot'], 'steelblue', linewidth = 1.5)
plt.plot(selected_h_5['V_more'],selected_h_5['Rtot'], 'mediumpurple', linewidth = 2.0, linestyle = 'dotted')
plt.plot(selected_h_5_5['V_more'],selected_h_5_5['Rtot'], 'mediumseagreen', linewidth = 1.0)
plt.plot(selected_h_8['V_more'],selected_h_8['Rtot'], 'darkviolet', linewidth = 1)
plt.plot(selected_h_10['V_more'],selected_h_10['Rtot'], 'hotpink', linewidth = 1)


plt.grid()
plt.xlim(0.5,5)
plt.ylim(0,200)
plt.legend(['h = 3m','h =3.3m','h = 4.2m','h = 5m','h = 5.5m','h = 8m','h = 10m'])
plt.xlabel('V (m/s)',fontsize=10,weight='bold',labelpad=3)
plt.ylabel('Total resistance (kN)',fontsize=10,weight='bold',labelpad=3)
plt.xticks( weight = 'bold')
plt.yticks( weight = 'bold')
plt.savefig('Rtot vs velocity original-data-erik.pdf', dpi=100, format='pdf') 
plt.title('M6, L = 85 m, B = 9.5 m, T = 2 m')

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 3.5')
selected_h_3_2 = plot_df.query('h_more == 3.25 & V_more <= 3.7')
#selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 3.9')
#selected_h_4 = plot_df.query('h_more == 4 & V_more <= 4')
selected_h_4_2 = plot_df.query('h_more == 4.25 & V_more <= 4.2')
selected_h_5 = plot_df.query('h_more == 5 & V_more <= 4.55')
selected_h_5_5= plot_df.query('h_more == 5.5 & V_more <= 4.77')
selected_h_8 = plot_df.query('h_more == 8 & V_more <= 4.9' )
selected_h_10 = plot_df.query('h_more == 10 & V_more <= 4.9')

plt.figure(dpi=300)
plt.figure(figsize=(8,7))
plt.plot(selected_h_3['V_more'],selected_h_3['Pd'], 'magenta', linewidth = 1.0)
plt.plot(selected_h_3_2['V_more'],selected_h_3_2['Pd'], 'aqua', linewidth = 1.0)
#plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pd'], 'skyblue', linewidth = 1.0)
#plt.plot(selected_h_4['V_more'],selected_h_4['Pd'], 'skyblue', linewidth = 1.0)
plt.plot(selected_h_4_2['V_more'],selected_h_4_2['Pd'], 'steelblue', linewidth = 1.5)
plt.plot(selected_h_5['V_more'],selected_h_5['Pd'], 'mediumpurple', linewidth = 2.0, linestyle = 'dotted')
plt.plot(selected_h_5_5['V_more'],selected_h_5_5['Pd'], 'mediumseagreen', linewidth = 1.0)
plt.plot(selected_h_8['V_more'],selected_h_8['Pd'], 'darkviolet', linewidth = 1)
plt.plot(selected_h_10['V_more'],selected_h_10['Pd'], 'hotpink', linewidth = 1)


plt.grid()
plt.xlim(0,5)
plt.ylim(0,800)
plt.legend(['h = 3m','h =3.3m','h = 4.2m','h = 5m','h = 5.5m','h = 8m','h = 10m'])
plt.xlabel('V (m/s)',fontsize=10,weight='bold',labelpad=3)
plt.ylabel('Delivered Power (kW)',fontsize=10,weight='bold',labelpad=3)
plt.xticks( weight = 'bold')
plt.yticks( weight = 'bold')
plt.savefig('Pd vs velocity original-data-erik.pdf', dpi=100, format='pdf') 
plt.title('M6, L = 85 m, B = 9.5 m, T = 2 m')

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4.5')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.5')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_3['V_more'],selected_h_3['Pe'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pe'], 'red', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Pe'], 'purple', linewidth = 2.5, linestyle = 'dotted')
plt.plot(selected_h_5['V_more'],selected_h_5['Pe'], 'purple', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Pe'], 'purple', linewidth = 1, linestyle = 'dashed')

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Pe (kW)')
plt.legend(['Pe~3m','Pe~3.5m','Pe~4m','Pe~5m','Pe~6m'])
plt.xlim(0, 5)
plt.ylim()
plt.grid()
plt.title('Pe');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 3.5')
selected_h_3_2 = plot_df.query('h_more == 3.25 & V_more <= 3.7')
#selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 3.9')
#selected_h_4 = plot_df.query('h_more == 4 & V_more <= 4')
selected_h_4_2 = plot_df.query('h_more == 4.25 & V_more <= 4.2')
selected_h_5 = plot_df.query('h_more == 5 & V_more <= 4.55')
selected_h_5_5= plot_df.query('h_more == 5.5 & V_more <= 4.77')
selected_h_8 = plot_df.query('h_more == 8 & V_more <= 4.9' )
selected_h_10 = plot_df.query('h_more == 10 & V_more <= 4.9')

plt.figure(dpi=300)
plt.figure(figsize=(8,7))
plt.plot(selected_h_3['V_more'],selected_h_3['Pd'], 'magenta', linewidth = 1.0)
plt.plot(selected_h_3_2['V_more'],selected_h_3_2['Pd'], 'aqua', linewidth = 1.0)
#plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pd'], 'skyblue', linewidth = 1.0)
#plt.plot(selected_h_4['V_more'],selected_h_4['Pd'], 'skyblue', linewidth = 1.0)
plt.plot(selected_h_4_2['V_more'],selected_h_4_2['Pd'], 'steelblue', linewidth = 1.5)
plt.plot(selected_h_5['V_more'],selected_h_5['Pd'], 'mediumpurple', linewidth = 2.0, linestyle = 'dotted')
plt.plot(selected_h_5_5['V_more'],selected_h_5_5['Pd'], 'mediumseagreen', linewidth = 1.0)
plt.plot(selected_h_8['V_more'],selected_h_8['Pd'], 'darkviolet', linewidth = 1)
plt.plot(selected_h_10['V_more'],selected_h_10['Pd'], 'hotpink', linewidth = 1)


plt.grid()
plt.xlim(0,5)
plt.ylim(0,800)
plt.legend(['h = 3m','h =3.3m','h = 4.2m','h = 5m','h = 5.5m','h = 8m','h = 10m'])
plt.xlabel('V (m/s)',fontsize=10,weight='bold',labelpad=3)
plt.ylabel('Delivered Power (kW)',fontsize=10,weight='bold',labelpad=3)
plt.xticks( weight = 'bold')
plt.yticks( weight = 'bold')
plt.savefig('Pd vs velocity original-data.pdf', dpi=100, format='pdf') 
plt.title('M6, L = 85 m, B = 9.5 m, T = 2 m')

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 3.5')
selected_h_3_2 = plot_df.query('h_more == 3.25 & V_more <= 3.7')
#selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 3.9')
#selected_h_4 = plot_df.query('h_more == 4 & V_more <= 4')
selected_h_4_2 = plot_df.query('h_more == 4.25 & V_more <= 4.2')
selected_h_5 = plot_df.query('h_more == 5 & V_more <= 4.55')
selected_h_5_5= plot_df.query('h_more == 5.5 & V_more <= 4.77')
selected_h_8 = plot_df.query('h_more == 8 & V_more <= 4.9' )
selected_h_10 = plot_df.query('h_more == 10 & V_more <= 4.9')

plt.figure(dpi=300)
plt.figure(figsize=(8,7))
plt.plot(selected_h_3['V_more'],selected_h_3['Rtot'], 'magenta', linewidth = 1.0)
plt.plot(selected_h_3_2['V_more'],selected_h_3_2['Rtotd'], 'aqua', linewidth = 1.0)
#plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pd'], 'skyblue', linewidth = 1.0)
#plt.plot(selected_h_4['V_more'],selected_h_4['Pd'], 'skyblue', linewidth = 1.0)
plt.plot(selected_h_4_2['V_more'],selected_h_4_2['Rtot'], 'steelblue', linewidth = 1.5)
plt.plot(selected_h_5['V_more'],selected_h_5['Pd'], 'mediumpurple', linewidth = 2.0, linestyle = 'dotted')
plt.plot(selected_h_5_5['V_more'],selected_h_5_5['Rtot'], 'mediumseagreen', linewidth = 1.0)
plt.plot(selected_h_8['V_more'],selected_h_8['Rtot'], 'darkviolet', linewidth = 1)
plt.plot(selected_h_10['V_more'],selected_h_10['Rtot'], 'hotpink', linewidth = 1)


plt.grid()
plt.xlim(0,5)
plt.ylim(0,200)
plt.legend(['h = 3m','h =3.3m','h = 4.2m','h = 5m','h = 5.5m','h = 8m','h = 10m'])
plt.xlabel('V (m/s)',fontsize=10,weight='bold',labelpad=3)
plt.ylabel('Total resistance (kN)',fontsize=10,weight='bold',labelpad=3)
plt.xticks( weight = 'bold')
plt.yticks( weight = 'bold')
plt.savefig('Rtot vs velocity Erikmethod-data.pdf', dpi=100, format='pdf') 
plt.title('M6, L = 85 m, B = 9.5 m, T = 2 m')